In [43]:
import random

# Словарь fallback-значений для нетерминалов, чтобы на максимальной глубине ветка завершалась корректно.

fallback_dict = {
    "expr": "1",
    "sum": "2",
    "product": "3",
    "power": "4",
    "postfix": "5",
    "primary": "6",
    "group": r"{x}",
    "frac_expr": r"{1}{2}",
    "integral_limits": "",
    "expr_opt": "",
    "limit_limits": "",
}

In [83]:
def generate_formula(
    grammar,
    weights,
    terminal_generators,
    symbol="start",
    depth=0,
    max_depth=10,
    recursion_bonus=0.5,
    default_weight=0.8,
):
    """
    Рекурсивная генерация формулы с фиксированной максимальной глубиной.
    Если достигнута максимальная глубина, для нетерминалов возвращаются
    заранее заданные fallback-значения, чтобы ветки завершались терминалами.

    :param grammar: словарь грамматики, где ключ – нетерминал, а значение – список альтернатив (каждая альтернатива – список токенов)
    :param weights: словарь весов для операторов и спец-токенов
    :param terminal_generators: словарь генераторов для терминальных символов
    :param symbol: текущий символ (нетерминал или терминал)
    :param depth: текущая глубина рекурсии
    :param max_depth: максимальная допустимая глубина рекурсии
    :param recursion_bonus: вес для рекурсивного вызова (если текущий символ встречается внутри своей же альтернативы)
    :param default_weight: вес по умолчанию для токенов, отсутствующих в weights
    :return: сгенерированная строка (формула)
    """
    # Если глубина превышает max_depth – возвращаем пустую строку.
    if depth > max_depth:
        return "", {}
    
    # Если мы на максимальной глубине и symbol – нетерминал, возвращаем fallback.
    if depth == max_depth and symbol in grammar:
        token = fallback_dict.get(symbol, "")
        return token, {token: 1}
    
    # Если symbol не является ключом грамматики, значит это терминал.
    if symbol not in grammar:
        if symbol in terminal_generators:
            token = terminal_generators[symbol]()
        else:
            token = symbol
        return token, {token: 1}

    alternatives = grammar[symbol]
    alt_weights = []
    for alt in alternatives:
        total_weight = 0
        for token in alt:
            if token == symbol:
                effective = recursion_bonus if depth < max_depth else default_weight
            else:
                effective = weights.get(token, default_weight)
            total_weight += effective
        alt_weights.append(total_weight)
    
    # Инвертированные веса: чем меньше сумма весов, тем выше вероятность выбора
    epsilon = 1e-6
    inv_weights = [1 / (w + epsilon) for w in alt_weights]
    total_inv = sum(inv_weights)
    probabilities = [w / total_inv for w in inv_weights]
    
    chosen_alt = random.choices(alternatives, weights=probabilities, k=1)[0]
    total_counts = {}
    result = ""
    for token in chosen_alt:
        # Если токен – нетерминал, то если следующая глубина равна max_depth, используем fallback,
        # иначе продолжаем рекурсию.
        if token in grammar:
            if depth + 1 == max_depth:
                result += fallback_dict.get(token, "")
            else:
                sub_formula, sub_counts = generate_formula(
                grammar,
                weights,
                terminal_generators,
                token,
                depth + 1,
                max_depth,
                recursion_bonus,
                default_weight,
                )
                result += sub_formula
                for k, v in sub_counts.items():
                    total_counts[k] = total_counts.get(k, 0) + v
        else:
            if token in terminal_generators:
                result += terminal_generators[token]()
            else:
                result += token
        
            
    return result, total_counts

In [84]:
grammar = {
    "start": [["expr"]],
    "expr": [["sum"]],
    "sum": [
        ["product"],
        ["(", "sum", ")", "product"],
        ["(", "sum", ")", "-", "product"],
    ],
    "product": [
        ["power"],
        ["product", "\\cdot", " ", "power"],
        ["product", "\\times", " ", "power"],
        ["product", "/", " ", "power"],
        ["product", "BINOP_FUNC", " ", "power"],
    ],
    "power": [
        ["{", "postfix", "}"],
        ["{", "postfix", "}", "^", "{", "power", "}"],
    ],
    "postfix": [
        ["{","primary", "}"],
        ["{","postfix", "}", "_", "{", "primary", "}"],
    ],
    "primary": [
        ["NUMBER"],
        ["GREEK"],
        ["LATIN"],
        ["CAPS_LATIN"],
        ["FRAC", "frac_expr"],
        ["BINOP_FUNC", "group"],
        ["FUNCTION", "group"],
        ["(", "expr", ")"],
        ["[", "expr", "]"],
        ["{", "expr", "}"],
        ["INTEGRAL", "integral_limits", " ", "expr_opt"],
        ["LIMIT", "limit_limits", " ", "expr"],
        ["|", "expr", "|"],
    ],
    "group": [["{", "expr", "}"]],
    "frac_expr": [["{", "expr", "}", "{", "expr", "}"]],
    "integral_limits": [
        [],
        ["_", "group"],
        ["^", "group"],
        ["_", "group", "^", "group"],
    ],
    "expr_opt": [
        [],
        ["expr"],
    ],
    "limit_limits": [
        [],
        ["_", "group"],
    ],
}

weights = {
    "+": 1.0,
    "-": 1.0,
    "\\cdot": 1.0,
    "/": 1.0,
    "BINOP_FUNC": 1.0,
    "^": 1.0,
    "_": 1.0,
    "LIMIT": 1.0,
    "INTEGRAL": 1.0,
    "FUNCTION": 1.0,
    "FRAC": 1.0,
}

terminal_generators = {
    "BINOP_FUNC": lambda: random.choice(["\\min", "\\max"]),
    "FUNCTION": lambda: random.choice(
        [
            "\\sin",
            "\\cos",
            "\\tan",
            "\\log",
            "\\ln",
            "\\exp",
            "\\sqrt",
            "\\arcsin",
            "\\arccos",
            "\\arctan",
        ]
    ),
    "FRAC": lambda: "\\frac",
    "NUMBER": lambda: (
        str(random.randint(1, 9))
        if random.random() < 0.8
        else str(random.randint(0, 9999)) + "." + str(random.randint(0, 9999))
    ),
    "GREEK": lambda: random.choice(
        ["\\alpha", "\\beta", "\\gamma", "\\delta", "\\epsilon"]
    ),
    "LATIN": lambda: "".join(
        random.choices("abcdefghijklmnopqrstuvwxyz", k=random.randint(1, 4))
    ),
    "CAPS_LATIN": lambda: "".join(
        random.choices("abcdefghijklmnopqrstuvwxyz".upper(), k=random.randint(1, 4))
    ),
    "INTEGRAL": lambda: "\\int",
    "LIMIT": lambda: "\\lim",
}

In [87]:
random.seed(42)
# Генерация нескольких формул с фиксированной глубиной
n = 1000
formulas = [
    generate_formula(grammar, weights, terminal_generators)
    for _ in range(n)
]
for formula in formulas:
    print(formula)

('{{\\int }}^{{{O}}}', {})
('{{{{EYI}}_{6}}_{\\frac{3}{3}}}', {})
('{{2}}\\times {{[2]}}\\cdot {{\\cos{2}}}/ {{rv}}^{{{5}}^{{{{\\log{x}}}_{\\max{1}}}}}', {})
('{{|1|}}\\times {{(2)}}\\cdot {{\\min{2}}}\\times {{R}}', {})
('(({{\\ln{1}}}){{{6}}_{\\gamma}}\\cdot {{(2)}}^{{{5038.3923}}}/ {{{{Z}}_{DM}}_{NPZ}}){{6}}\\times {{\\alpha}}^{{{6}}}\\times {{4}}/ {{hm}}\\min {{{\\delta}}_{[4]}}^{{{\\cos{2}}}}', {})
('{{{(2)3}}}\\cdot {{\\min{(2)3}}}^{{{\\delta}}}', {})
('{{{\\arccos{2}}}_{\\beta}}^{{{BPN}}^{{{|2|}}^{{{\\min{x}}}}}}', {})
('{{\\frac{2}{2}}}\\cdot {{8}}', {})
('{{{\\gamma}}_{2}}^{{{\\epsilon}}^{{{|2|}}^{{{{6}}_{\\min{x}}}}}}', {})
('({{\\ln{2}}}){{{qg}}_{\\arccos{2}}}\\min {{{{[2]}}_{\\beta}}_{1}}', {})
('{{{\\frac{2}{2}}}_{|(3)-3\\times 4|}}^{{{{{{5}_{6}}_{\\alpha}}_{w}}_{\\delta}}}', {})
('({{(2)}}\\times {{{3}}})-{{{\\frac{1}{1}}}_{((2)-3)}}\\min {{\\delta}}', {})
('{{rdx}}\\times {{oer}}\\times {{{\\frac{1}{1}}}_{[3]}}\\times {{lfmx}}^{{{6}}^{{{{db}}_{|2|}}}}', {})
('({{\\cos{1}

In [73]:
from IPython.display import display, Math
display(Math(formulas[10]))

<IPython.core.display.Math object>

In [71]:
display(Math(r"\frac{\,\,}{}"))

<IPython.core.display.Math object>